In [132]:
import wptools
import re
import datetime

### Функция для получения участников группы, нуждается в доработке

In [133]:
def get_members(band):
    so = wptools.page(band).get_parse()
    infobox = so.data['infobox']
    try:
        infobox = infobox['current_members']
        
        if 'Virtual' in infobox:
            infobox = infobox.replace("'","").replace(':','').replace('[','').replace(']','')
            infobox = infobox.split('<br>')
            index = infobox.index('Non-virtual')
            infobox = infobox[index+1:]
            
        else:
            if '<br>' in infobox:
                infobox = infobox.replace("'","").replace(':','').replace('[','').replace(']','').replace('{{plainlist','')
                infobox1 = infobox.split('<br>')
            else:
                infobox = infobox.replace("'","").replace(':','').replace('[','').replace(']','').replace('*','').replace('{{plainlist','')
                if '\n' in infobox:
                    infobox1 = infobox.split('\n')
                elif '<br/>' in infobox:
                    infobox1 = infobox.split('<br/>')
                else:
                    infobox1 = infobox.split('<br />')
            infobox = []
            for k in infobox1:
                infobox.append(k.split('|')[0])
    
    except:
        try:
            infobox = infobox['past_members']
            infobox = infobox.replace("'","").replace(':','').replace('[','').replace(']','').replace('*','')
            if '\n' in infobox:
                infobox1 = infobox.split('\n')
            elif '<br>' in infobox:
                infobox1 = infobox.split('<br>')
            else:
                infobox1 = infobox.split('<br />')
            infobox = []
            for k in infobox1:
                infobox.append(k.split('|')[0])
        except:
            infobox = ['nothing found']
    if len(infobox) > 1:
        return [value for value in infobox if value]
    else:
        return infobox[0]

### Функция для получения групп, с которыми сотрудничал исполнитель или артист

In [134]:
def get_bands(artist):
    so = wptools.page(artist).get_parse()
    infobox = so.data['infobox']
    try:
        infobox = infobox['module']
        infobox = infobox.replace("'","").replace(':','').replace('[','').replace(']','').replace('*','').replace('nowrap','')
        infobox = infobox.replace("flatlist","").replace('{','').replace('}','').replace('|','').replace('=','')
        
        infobox = infobox.split('\n')
        for el in infobox:
            if 'associated_acts' in el:
                need = el
        index1 = infobox.index(need)
        infobox1 = infobox[index1+1:]
        
        infobox = []
        for k in infobox1:
            if k != '':
                k = k.split(')')
                if '(' in k[0]:
                    infobox.append(k[0]+')')
                else:
                    infobox.append(k[0])
            else:
                break
        
    except:
        try:
            infobox = infobox['associated_acts']
            
        except:
            infobox = 'nothing found'
    return infobox

### Функция для получения возраста и места рождения

In [135]:
def get_data_place(artist):
    so = wptools.page(artist).get_parse()
    infobox = so.data['infobox']
    try:
        birth_date = infobox['birth_date']
        nums = re.findall(r'\d+', birth_date)
        date = datetime.datetime(int(nums[0]), int(nums[1]), int(nums[2]))
        now = datetime.datetime.now()
        age = round((now-date).days/365.25)
    except: 
        print(infobox)
        age = 'None'
    try:
        birth_place = infobox['birth_place'].split(',')[-1]
    except:
        birth_place = 'None'
    return birth_place, age

##### Мы написали функции для получения данных, теперь наполним наши данные.
Начнем с Gorillaz.

In [136]:
gorillaz_members = []
for member in get_members('Gorillaz'):
    d = {}
    d['name'] = member
    d['related'] = 'Gorillaz'
    gorillaz_members.append(d)

en.wikipedia.org (parse) Gorillaz
en.wikipedia.org (imageinfo) File:Gorillaz Song Machine Promo Art.jpg
Gorillaz (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Gorillaz ...
  infobox: <dict(11)> name, image, caption, origin, genre, years_a...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/category:G...
  pageid: 13084
  parsetree: <str(153855)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Gorillaz
  wikibase: Q189991
  wikidata_url: https://www.wikidata.org/wiki/Q189991
  wikitext: <str(129612)> {{Short description|British virtual band...
}


In [137]:
gorillaz_members

[{'name': 'Damon Albarn', 'related': 'Gorillaz'},
 {'name': 'Jamie Hewlett', 'related': 'Gorillaz'},
 {'name': 'Remi Kabaka Jr.', 'related': 'Gorillaz'}]

In [138]:
def get_bands2(members):
    dicts = []
    for member in members:
        bands = get_bands(member['name'])
        if bands != 'nothing found':
            for el in bands:
                d = {}
                d['name'] = member['name']
                d['related'] = el
                dicts.append(d)
        else:
            pass
    
    return dicts
        

In [139]:
gorillaz_members2 = get_bands2(gorillaz_members)

en.wikipedia.org (parse) Damon Albarn
en.wikipedia.org (imageinfo) File:Roskilde Festival Gorillaz-4 (c...
Damon Albarn (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Roskilde ...
  infobox: <dict(12)> name, honorific_suffix, image, caption, birt...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:D...
  pageid: 100097
  parsetree: <str(110521)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Damon Albarn
  wikibase: Q272069
  wikidata_url: https://www.wikidata.org/wiki/Q272069
  wikitext: <str(89135)> {{Short description|British musician (bor...
}
en.wikipedia.org (parse) Jamie Hewlett
en.wikipedia.org (imageinfo) File:Jamie Hewlett in 2014 signing c...
Jamie Hewlett (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Jamie Hew...
  infobox: <dict(18)> image, caption, birth_name, birth_date, birt...
  pageid: 512979
  parsetree: <str(27933)> <root><template><title>Short description...
  r

In [140]:
def get_members2(bands):
    list_of_dicts = []
    for band in bands:
        members = get_members(band['related'])
        if members != 'nothing found':
            for member in members:
                d = {}
                d['name'] = member
                d['related'] = band['related']
                list_of_dicts.append(d)
        else:
            pass
    
    return list_of_dicts
        

In [141]:
members = get_members2(gorillaz_members2)

en.wikipedia.org (parse)  Blur (band)
en.wikipedia.org (imageinfo) File:Blur (band).png
Blur (band) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Blur (ban...
  infobox: <dict(9)> name, image, caption, origin, genre, years_ac...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:B...
  pageid: 43007
  parsetree: <str(56552)> <root><template><title>Short description...
  requests: <list(2)> parse, imageinfo
  title: Blur (band)
  wikibase: Q485820
  wikidata_url: https://www.wikidata.org/wiki/Q485820
  wikitext: <str(43618)> {{Short description|English rock band}}{{...
}
en.wikipedia.org (parse)  Gorillaz
en.wikipedia.org (imageinfo) File:Gorillaz Song Machine Promo Art.jpg
Gorillaz (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Gorillaz ...
  infobox: <dict(11)> name, image, caption, origin, genre, years_a...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/category:G...
  pageid: 13084
  parsetree: <str(153855)> <root><

Так можно делать до бесконечности, поэтому мы пока остановимся на тех данных что у нас есть.

In [142]:
everything = gorillaz_members + gorillaz_members2 + members

In [143]:
everything

[{'name': 'Damon Albarn', 'related': 'Gorillaz'},
 {'name': 'Jamie Hewlett', 'related': 'Gorillaz'},
 {'name': 'Remi Kabaka Jr.', 'related': 'Gorillaz'},
 {'name': 'Damon Albarn', 'related': ' Blur (band)'},
 {'name': 'Damon Albarn', 'related': ' Gorillaz'},
 {'name': 'Damon Albarn', 'related': ' Monkey Journey to the West'},
 {'name': 'Damon Albarn', 'related': ' The Good, the Bad & the Queen'},
 {'name': 'Damon Albarn', 'related': ' Mali Music'},
 {'name': 'Damon Albarn', 'related': ' Rocket Juice & the Moon'},
 {'name': 'Damon Albarn', 'related': ' DRC Music'},
 {'name': 'Damon Albarn', 'related': ' Massive Attack'},
 {'name': 'Damon Albarn', 'related': ' The Black Ghosts'},
 {'name': 'Damon Albarn', 'related': ' Africa Express (organization)'},
 {'name': ' Damon Albarn', 'related': ' Blur (band)'},
 {'name': ' Graham Coxon', 'related': ' Blur (band)'},
 {'name': ' Alex James (musician)', 'related': ' Blur (band)'},
 {'name': ' Dave Rowntree', 'related': ' Blur (band)'},
 {'name': '

In [144]:
import json

with open("/Users/ico/Desktop/базы данных/data.json", "w", encoding="utf-8") as file:
    for el in everything:
        try:
            json.dump(el['name'] + '\t' + el['related'] + '\t' +str(get_data_place(el['name'])[0]) + '\t' \
                      + str(get_data_place(el['name'])[1]) + '\n', file)
        except:
            pass

en.wikipedia.org (parse) Damon Albarn
en.wikipedia.org (imageinfo) File:Roskilde Festival Gorillaz-4 (c...
Damon Albarn (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Roskilde ...
  infobox: <dict(12)> name, honorific_suffix, image, caption, birt...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:D...
  pageid: 100097
  parsetree: <str(110521)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Damon Albarn
  wikibase: Q272069
  wikidata_url: https://www.wikidata.org/wiki/Q272069
  wikitext: <str(89135)> {{Short description|British musician (bor...
}
en.wikipedia.org (parse) Damon Albarn
en.wikipedia.org (imageinfo) File:Roskilde Festival Gorillaz-4 (c...
Damon Albarn (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Roskilde ...
  infobox: <dict(12)> name, honorific_suffix, image, caption, birt...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:D...
  pageid: 100097
  par

en.wikipedia.org (imageinfo) File:Roskilde Festival Gorillaz-4 (c...
Damon Albarn (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Roskilde ...
  infobox: <dict(12)> name, honorific_suffix, image, caption, birt...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:D...
  pageid: 100097
  parsetree: <str(110521)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Damon Albarn
  wikibase: Q272069
  wikidata_url: https://www.wikidata.org/wiki/Q272069
  wikitext: <str(89135)> {{Short description|British musician (bor...
}
en.wikipedia.org (parse) Damon Albarn
en.wikipedia.org (imageinfo) File:Roskilde Festival Gorillaz-4 (c...
Damon Albarn (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Roskilde ...
  infobox: <dict(12)> name, honorific_suffix, image, caption, birt...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:D...
  pageid: 100097
  parsetree: <str(110521)> <root><template>

en.wikipedia.org (imageinfo) File:Roskilde Festival Gorillaz-4 (c...
Damon Albarn (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Roskilde ...
  infobox: <dict(12)> name, honorific_suffix, image, caption, birt...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:D...
  pageid: 100097
  parsetree: <str(110521)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Damon Albarn
  wikibase: Q272069
  wikidata_url: https://www.wikidata.org/wiki/Q272069
  wikitext: <str(89135)> {{Short description|British musician (bor...
}
en.wikipedia.org (parse)  Graham Coxon
en.wikipedia.org (imageinfo) File:Graham Coxon @ Falmouth 1.jpg
Graham Coxon (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Graham Co...
  infobox: <dict(15)> name, background, image, caption, birth_name...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:G...
  pageid: 294084
  parsetree: <str(28338)> <root><template><titl

en.wikipedia.org (imageinfo) File:Roskilde Festival Gorillaz-4 (c...
Damon Albarn (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Roskilde ...
  infobox: <dict(12)> name, honorific_suffix, image, caption, birt...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:D...
  pageid: 100097
  parsetree: <str(110521)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Damon Albarn
  wikibase: Q272069
  wikidata_url: https://www.wikidata.org/wiki/Q272069
  wikitext: <str(89135)> {{Short description|British musician (bor...
}
en.wikipedia.org (parse)  Paul Simonon
en.wikipedia.org (imageinfo) File:Paul Simonon mg 6692 crop.jpg
Paul Simonon (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Paul Simo...
  infobox: <dict(13)> name, image, caption, background, birth_name...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Paul_Simon...
  pageid: 623467
  parsetree: <str(25756)> <root><template><titl

{'name': 'Rodaidh McDonald', 'birth_place': '[[Edinburgh]], Scotland', 'occupation': 'Music producer, record company executive', 'label': '[[XL Recordings|XL]]', 'associated_acts': '{{flatlist|\n*[[The xx]]\n*[[Sampha]]\n*[[King Krule]]\n*[[Savages (band)|Savages]]\n*[[Låpsley]]\n*[[Daughter (band)|Daughter]]\n*[[Adele]]\n*[[Vampire Weekend]]\n*[[Jamie xx]]\n*[[David Byrne]]\n*[[Gil Scott Heron]]}}', 'website': '{{url|rodaidhmcdonald.com}}'}


...

Theo Keating (en) data
{
  infobox: <dict(10)> name, background, alias, origin, instrument,...
  pageid: 6938540
  parsetree: <str(10772)> <root><template><title>Short description...
  requests: <list(1)> parse
  title: Theo Keating
  wikibase: Q727966
  wikidata_url: https://www.wikidata.org/wiki/Q727966
  wikitext: <str(7532)> {{Short description|British DJ, musician a...
}
en.wikipedia.org (parse) Simon William Lord


{'name': 'Theo Keating', 'background': 'non_vocal_instrumentalist', 'alias': 'Fake Blood, Touché', 'origin': 'London, England', 'instrument': '[[Phonograph|Turntables]], programming', 'genre': '[[Electro house]], [[fidget house]], [[Hip hop music|hip hop]], [[big beat]]', 'occupation': 'DJ, musician, producer', 'label': 'Blood Music, PIAS, [[Southern Fried Records]], Cheap Thrills', 'associated_acts': '[[Fake Blood]], [[The Black Ghosts]], [[The Wiseguys]]', 'website': '{{url|facebook.com/iamfakeblood}}'}


Simon William Lord (en) data
{
  infobox: <dict(7)> name, background, birth_place, genre, occupat...
  pageid: 25568105
  parsetree: <str(6555)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Simon William Lord
  wikibase: Q7520050
  wikidata_url: https://www.wikidata.org/wiki/Q7520050
  wikitext: <str(4688)> {{Use dmy dates|date=September 2014}}{{Inf...
}
en.wikipedia.org (parse) Simon William Lord


{'name': 'Simon William Lord', 'background': 'non_performing_personnel', 'birth_place': 'London, England', 'genre': '[[Electronic music|Electronic]], [[indie rock]], [[synthpop]], [[electro house]]', 'occupation': 'Record producer, songwriter, musician, [[remix]]er', 'years_active': '2000–present', 'associated_acts': '[[Simian (band)|Simian]]<br/>[[The Black Ghosts]]<br/> [[Lord Skywave]]<br/> Garden<br/> [[Dillon Francis]]'}
{'name': 'Simon William Lord', 'background': 'non_performing_personnel', 'birth_place': 'London, England', 'genre': '[[Electronic music|Electronic]], [[indie rock]], [[synthpop]], [[electro house]]', 'occupation': 'Record producer, songwriter, musician, [[remix]]er', 'years_active': '2000–present', 'associated_acts': '[[Simian (band)|Simian]]<br/>[[The Black Ghosts]]<br/> [[Lord Skywave]]<br/> Garden<br/> [[Dillon Francis]]'}


Simon William Lord (en) data
{
  infobox: <dict(7)> name, background, birth_place, genre, occupat...
  pageid: 25568105
  parsetree: <str(6555)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Simon William Lord
  wikibase: Q7520050
  wikidata_url: https://www.wikidata.org/wiki/Q7520050
  wikitext: <str(4688)> {{Use dmy dates|date=September 2014}}{{Inf...
}
